# Data Wrangling with Spark

I use this notebook firts to make some knowledge about the dataset.

In [1]:
import findspark
findspark.init()

import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql import functions as F

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [3]:
####### Setting SparkSession:

spark = SparkSession \
            .builder \
            .appName("Wrangling Data") \
            .getOrCreate()

spark

## Read the dataset files:
|

|--> Item information

|

|--> Station information

|

|--> Measurement info

|

|--> Measurement Summary


In [4]:
measurement_item_file  = os.path.join('Resources', "Data_Measurement_item_info.csv")
measurement_station_file  = os.path.join('Resources', "Data_Measurement_station_info.csv")
measurement_info_file  = os.path.join('Resources', "Measurement_info.csv")
measurement_file  = os.path.join('Resources', "Measurement_summary.csv")

### Item dataset

In [5]:
item_info = spark.read.format("csv").option("header", "true").load(measurement_item_file)

In [6]:
item_info.printSchema()

root
 |-- Item code: string (nullable = true)
 |-- Item name: string (nullable = true)
 |-- Unit of measurement: string (nullable = true)
 |-- Good(Blue): string (nullable = true)
 |-- Normal(Green): string (nullable = true)
 |-- Bad(Yellow): string (nullable = true)
 |-- Very bad(Red): string (nullable = true)



### Station dataset

In [7]:
station_info = spark.read.format("csv").option("header", "true").load(measurement_station_file)
station_info.printSchema()

root
 |-- Station code: string (nullable = true)
 |-- Station name(district): string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



### Info dataset

In [8]:
info = spark.read.format("csv").option("header", "true").load(measurement_info_file)
info.printSchema()

root
 |-- Measurement date: string (nullable = true)
 |-- Station code: string (nullable = true)
 |-- Item code: string (nullable = true)
 |-- Average value: string (nullable = true)
 |-- Instrument status: string (nullable = true)



### Measurament dataset

In [9]:
measurament = spark.read.format("csv").option("header", "true").load(measurement_file)
measurament.printSchema()

root
 |-- Measurement date: string (nullable = true)
 |-- Station code: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- SO2: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- O3: string (nullable = true)
 |-- CO: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- PM2.5: string (nullable = true)



In [10]:
measurament.show(5)

+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+-----+
|Measurement date|Station code|             Address|  Latitude|  Longitude|  SO2|  NO2|   O3| CO|PM10|PM2.5|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+-----+
|2017-01-01 00:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.059|0.002|1.2|73.0| 57.0|
|2017-01-01 01:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.058|0.002|1.2|71.0| 59.0|
|2017-01-01 02:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0| 59.0|
|2017-01-01 03:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.004|0.056|0.002|1.2|70.0| 58.0|
|2017-01-01 04:00|         101|19, Jong-ro 35ga-...|37.5720164|127.0050075|0.003|0.051|0.002|1.2|69.0| 61.0|
+----------------+------------+--------------------+----------+-----------+-----+-----+-----+---+----+-----+
only showing top 5 

In [11]:
item_info.show()

+---------+---------+-------------------+----------+-------------+-----------+-------------+
|Item code|Item name|Unit of measurement|Good(Blue)|Normal(Green)|Bad(Yellow)|Very bad(Red)|
+---------+---------+-------------------+----------+-------------+-----------+-------------+
|        1|      SO2|                ppm|      0.02|         0.05|       0.15|          1.0|
|        3|      NO2|                ppm|      0.03|         0.06|        0.2|          2.0|
|        5|       CO|                ppm|       2.0|          9.0|       15.0|         50.0|
|        6|       O3|                ppm|      0.03|         0.09|       0.15|          0.5|
|        8|     PM10|      Mircrogram/m3|      30.0|         80.0|      150.0|        600.0|
|        9|    PM2.5|      Mircrogram/m3|      15.0|         35.0|       75.0|        500.0|
+---------+---------+-------------------+----------+-------------+-----------+-------------+



## Info - Class

In [12]:
class equipament:
    
    def __init__(self, code,name,unit,level1,level2,level3,level4):
        self.code = code
        self.name = name
        self.unit = unit
        self.good = float(level1)
        self.normal = float(level2)
        self.bad = float(level3)
        self.veryBad = float(level4)
        self.color = 0
    
    
    
    def getLevel(self,value):
        if value < self.good:
            self.color = 1
            return ("Good")
        elif value < self.normal:
            self.color = 2
            return ("Normal")
        elif value < self.bad:
            self.color = 3
            return ("Bad")
        elif value < self.veryBad:
            self.color = 4
            return ("Very Bad")
        
    def __repr__(self):
        return(f"Equipament Code = {str(self.code)}, Name = {str(self.name)}, Unit of Measurement = {str(self.unit)}")

In [13]:
item_df = item_info.toPandas()

list_equipaments = [equipament(row[0],row[1],row[2],row[3],row[4],row[5],row[6]) for index,row in item_df.iterrows()]

In [14]:
equipaments = {
    "SO2": list_equipaments[0],
    "NO2": list_equipaments[1],
    "CO": list_equipaments[2],
    "O3": list_equipaments[3],
    "PM10": list_equipaments[4],
    "PM2.5": list_equipaments[5]
}
equipaments

{'SO2': Equipament Code = 1, Name = SO2, Unit of Measurement = ppm,
 'NO2': Equipament Code = 3, Name = NO2, Unit of Measurement = ppm,
 'CO': Equipament Code = 5, Name = CO, Unit of Measurement = ppm,
 'O3': Equipament Code = 6, Name = O3, Unit of Measurement = ppm,
 'PM10': Equipament Code = 8, Name = PM10, Unit of Measurement = Mircrogram/m3,
 'PM2.5': Equipament Code = 9, Name = PM2.5, Unit of Measurement = Mircrogram/m3}

In [15]:
# Test

equipaments["SO2"].getLevel(0.0001)

'Good'

------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------

### Verify The Color Level of each measurement info

In [16]:
# Get the level Risk for SO2
level_risk = udf(lambda x: equipaments["SO2"].getLevel(float(x)))

In [17]:
# Pick a Type of Equipament Measurement
from pyspark.sql.functions import col

recent_measurements = measurament.select([col("SO2").alias("SO2"),col("Measurement date"),
                                          (F.regexp_extract('Measurement date',r'2017-01-01*', 0)).alias("DataNow")
                                         ])

# Adding a column with Level Risk


measurement_info_risk = recent_measurements.withColumn("Level", level_risk("SO2"))



In [18]:
measurement_info_risk.head(10)

[Row(SO2='0.004', Measurement date='2017-01-01 00:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.004', Measurement date='2017-01-01 01:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.004', Measurement date='2017-01-01 02:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.004', Measurement date='2017-01-01 03:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.003', Measurement date='2017-01-01 04:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.003', Measurement date='2017-01-01 05:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.003', Measurement date='2017-01-01 06:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.003', Measurement date='2017-01-01 07:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.004', Measurement date='2017-01-01 08:00', DataNow='2017-01-01', Level='Good'),
 Row(SO2='0.003', Measurement date='2017-01-01 09:00', DataNow='2017-01-01', Level='Good')]

### 1.) Question There is any Bad or Very Bad level in 2017-01-01, for SO2 , check This

In [19]:
measurament_now = measurament.select(["SO2","Measurement date","Station code"]) \
                            .filter(measurament["Measurement date"].rlike("2017-01-01*")) \


In [20]:
measurament_now = measurament_now.withColumn("Level", level_risk("SO2"))
measurament_now.show()

+-----+----------------+------------+-----+
|  SO2|Measurement date|Station code|Level|
+-----+----------------+------------+-----+
|0.004|2017-01-01 00:00|         101| Good|
|0.004|2017-01-01 01:00|         101| Good|
|0.004|2017-01-01 02:00|         101| Good|
|0.004|2017-01-01 03:00|         101| Good|
|0.003|2017-01-01 04:00|         101| Good|
|0.003|2017-01-01 05:00|         101| Good|
|0.003|2017-01-01 06:00|         101| Good|
|0.003|2017-01-01 07:00|         101| Good|
|0.004|2017-01-01 08:00|         101| Good|
|0.003|2017-01-01 09:00|         101| Good|
|0.004|2017-01-01 10:00|         101| Good|
|0.004|2017-01-01 11:00|         101| Good|
|0.004|2017-01-01 12:00|         101| Good|
|0.005|2017-01-01 13:00|         101| Good|
|0.006|2017-01-01 14:00|         101| Good|
|0.006|2017-01-01 15:00|         101| Good|
|0.006|2017-01-01 16:00|         101| Good|
|0.005|2017-01-01 17:00|         101| Good|
|0.005|2017-01-01 18:00|         101| Good|
|0.005|2017-01-01 19:00|        

In [21]:
bad_ocurence = measurament_now.filter(measurament_now["Level"] == "Bad").count()
very_bad_occurence = measurament_now.filter(measurament_now["Level"] == "Very Bad").count()

print(f"Numbers of Bad ocurrence = {bad_ocurence} and Numbers of Very Bad ocurrence = {very_bad_occurence}")

Numbers of Bad ocurrence = 2 and Numbers of Very Bad ocurrence = 0


### 2.) Station Code 102, measuremeants

In [22]:
station_info.show(2)

+------------+----------------------+--------------------+------------------+------------------+
|Station code|Station name(district)|             Address|          Latitude|         Longitude|
+------------+----------------------+--------------------+------------------+------------------+
|         101|             Jongno-gu|19, Jong-ro 35ga-...|37.572016399999995|127.00500749999999|
|         102|               Jung-gu|15, Deoksugung-gi...|37.564262899999996|126.97467569999999|
+------------+----------------------+--------------------+------------------+------------------+
only showing top 2 rows



In [23]:
measurament.select(["Station Code","SO2","Measurement date"]).filter(measurament["Station Code"] == 102)

DataFrame[Station Code: string, SO2: string, Measurement date: string]

--------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

## 3) Which station detected the highest amount of levels (Bad or Very bad) on 01-01-2017 for SO2

In [24]:
measurament_now = measurament.select(["Station Code","SO2","Measurement date"]) \
            .filter(measurament["Measurement date"].rlike("2017-01-01*")) \
            .withColumn("Level", level_risk("SO2"))
    
measurament_now.show()

+------------+-----+----------------+-----+
|Station Code|  SO2|Measurement date|Level|
+------------+-----+----------------+-----+
|         101|0.004|2017-01-01 00:00| Good|
|         101|0.004|2017-01-01 01:00| Good|
|         101|0.004|2017-01-01 02:00| Good|
|         101|0.004|2017-01-01 03:00| Good|
|         101|0.003|2017-01-01 04:00| Good|
|         101|0.003|2017-01-01 05:00| Good|
|         101|0.003|2017-01-01 06:00| Good|
|         101|0.003|2017-01-01 07:00| Good|
|         101|0.004|2017-01-01 08:00| Good|
|         101|0.003|2017-01-01 09:00| Good|
|         101|0.004|2017-01-01 10:00| Good|
|         101|0.004|2017-01-01 11:00| Good|
|         101|0.004|2017-01-01 12:00| Good|
|         101|0.005|2017-01-01 13:00| Good|
|         101|0.006|2017-01-01 14:00| Good|
|         101|0.006|2017-01-01 15:00| Good|
|         101|0.006|2017-01-01 16:00| Good|
|         101|0.005|2017-01-01 17:00| Good|
|         101|0.005|2017-01-01 18:00| Good|
|         101|0.005|2017-01-01 1

In [25]:
# Agregate every station code per Level

measurament_now_grouped = measurament_now.groupBy("Station Code").agg(F.collect_list("Level").alias("Levels"))

measurament_now_grouped.show()

+------------+--------------------+
|Station Code|              Levels|
+------------+--------------------+
|         125|[Good, Good, Good...|
|         124|[Good, Good, Good...|
|         101|[Good, Good, Good...|
|         112|[Good, Good, Good...|
|         113|[Good, Good, Good...|
|         107|[Good, Good, Good...|
|         110|[Good, Good, Good...|
|         120|[Good, Good, Good...|
|         118|[Good, Good, Good...|
|         104|[Good, Good, Good...|
|         102|[Good, Good, Good...|
|         111|[Good, Good, Good...|
|         103|[Good, Good, Good...|
|         115|[Good, Good, Good...|
|         122|[Good, Good, Good...|
|         108|[Good, Good, Good...|
|         117|[Good, Good, Good...|
|         114|[Good, Good, Good...|
|         106|[Good, Good, Good...|
|         116|[Good, Good, Good...|
+------------+--------------------+
only showing top 20 rows



In [26]:
measurament_now.filter((measurament_now["Level"] == "Bad") | (measurament_now["Level"] == "Very Bad")) \
               .groupBy("Station Code").agg(F.countDistinct("Level"))\
               .show() 

+------------+------------+
|Station Code|count(Level)|
+------------+------------+
|         112|           1|
|         120|           1|
+------------+------------+



In [27]:
measurament_now_grouped.select("Station Code",F.array_distinct(measurament_now_grouped.Levels).alias("Levls")).show()

+------------+-------------------+
|Station Code|              Levls|
+------------+-------------------+
|         125|             [Good]|
|         124|             [Good]|
|         101|             [Good]|
|         112|        [Good, Bad]|
|         113|             [Good]|
|         107|             [Good]|
|         110|             [Good]|
|         120|[Good, Normal, Bad]|
|         118|             [Good]|
|         104|             [Good]|
|         102|     [Good, Normal]|
|         111|             [Good]|
|         103|             [Good]|
|         115|             [Good]|
|         122|             [Good]|
|         108|             [Good]|
|         117|             [Good]|
|         114|             [Good]|
|         106|             [Good]|
|         116|             [Good]|
+------------+-------------------+
only showing top 20 rows



## 4) Location for SO2

In [29]:
station_info.show(5)

+------------+----------------------+--------------------+------------------+------------------+
|Station code|Station name(district)|             Address|          Latitude|         Longitude|
+------------+----------------------+--------------------+------------------+------------------+
|         101|             Jongno-gu|19, Jong-ro 35ga-...|37.572016399999995|127.00500749999999|
|         102|               Jung-gu|15, Deoksugung-gi...|37.564262899999996|126.97467569999999|
|         103|            Yongsan-gu|136, Hannam-daero...|37.540032700000005|         127.00485|
|         104|          Eunpyeong-gu|215, Jinheung-ro,...|        37.6098232|       126.9348476|
|         105|          Seodaemun-gu|32, Segeomjeong-r...|        37.5937421|       126.9496787|
+------------+----------------------+--------------------+------------------+------------------+
only showing top 5 rows



In [30]:
measurament_now.show(2)

+------------+-----+----------------+-----+
|Station Code|  SO2|Measurement date|Level|
+------------+-----+----------------+-----+
|         101|0.004|2017-01-01 00:00| Good|
|         101|0.004|2017-01-01 01:00| Good|
+------------+-----+----------------+-----+
only showing top 2 rows



In [43]:
measuramente_station = measurament_now.select(["SO2","Station Code"])\
                                        .groupBy("Station Code")\
                                        .agg(F.avg("SO2").alias("SO2"))\
                                        .withColumn("Level", level_risk("SO2"))

In [44]:
measuramente_station.show(10)

+------------+--------------------+-----+
|Station Code|                 SO2|Level|
+------------+--------------------+-----+
|         125|-2.26851851851848E-4| Good|
|         124|0.005370370370370368| Good|
|         101|0.004504629629629633| Good|
|         112|0.005851851851851843| Good|
|         113|-0.02700925925925...| Good|
|         107| 0.00466666666666667| Good|
|         110|0.005060185185185185| Good|
|         120| 0.00506481481481481| Good|
|         118|0.004712962962962965| Good|
|         104|0.004574074074074078| Good|
+------------+--------------------+-----+
only showing top 10 rows



In [46]:
## Convert to pandas to Plot

station_pandas = measuramente_station.toPandas()

In [48]:
station_pandas.head()

,Station Code,SO2,Level
0,125,-0.000227,Good
1,124,0.005370,Good
2,101,0.004505,Good
3,112,0.005852,Good
4,113,-0.027009,Good
